In [ ]:
# IPython and standard packages
import numpy as np
import matplotlib.pyplot as plt
from sympy.physics.vector import init_vprinting
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
InteractiveShell.ast_node_interactivity = "all"
get_ipython().run_line_magic('matplotlib', 'inline')
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')
init_vprinting(use_latex='mathjax', pretty_print=False)
%matplotlib inline
# Sympy
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols
# Yams sympy
from welib.yams.yams_sympy       import YAMSRigidBody, YAMSInertialBody, YAMSFlexibleBody
from welib.yams.yams_sympy_model import YAMSModel

# Inverted (rod) pendulum on a cart

 Point pendulum
 
```
 Coordinate system:
 
      
^ y     __   pendulum of mass m_p length l
|       \ \   
|        \ \  
|         \ \~|  angle theta positive about z
|   - c -  \_\|
O-- - k - --[ C ] -------> x 
            cart of mass m_c
```

In [ ]:
mp, mc, k, c, kp, cp, l = sp.symbols('m_p, m_c, k, c, k_p, c_p, l')
gravity = sp.symbols('g')
J_G = sp.symbols('J_G')
J_O = sp.symbols('J_O')
time = dynamicsymbols._t
x, theta = dynamicsymbols('x,theta')
xd, thetad = dynamicsymbols('xd,thetad')
Fx = dynamicsymbols('F_x')

In [ ]:
# --- Bodies, rigid body with COG displaced by d
ref = YAMSInertialBody('E') 
cart = YAMSRigidBody('C', mass=mc, rho_G=(0,0,0))
pend = YAMSRigidBody('P', mass=mp, rho_G=(0,l/2,0), J_G =(0,0,J_G)) # NOTE: Inertia defined at COG by default. 
bodies=[cart, pend] 
# --- Connections
ref.connectTo(cart, type='Free', rel_pos=(x, 0, 0))
cart.connectTo(pend, type='Joint', rel_pos=(0, 0, 0), rot_type='Body', rot_amounts=(0,0,theta), rot_order='XYZ')
# --- DOFs and kinetic equations
coordinates = [x,theta]
speeds      = [xd,thetad]
kdeqsSubs = []
for q,qd in zip(coordinates, speeds):
    kdeqsSubs += [(qd, sp.diff(q, time))]
# --- Loads
body_loads = []
for bdy in bodies: # gravity
    body_loads  += [(bdy, (bdy.masscenter, -bdy.mass * gravity * ref.frame.y))] # Watch out for convention here
body_loads += [(cart, (cart.masscenter,  - k * x * cart.frame.x - c * sp.diff(x,time) * cart.frame.x))] # stiffness/damping on chard
body_loads += [(cart, (cart.masscenter,  Fx * cart.frame.x))] # control/exteral force
body_loads += [(pend, (pend.frame,  -kp * theta * ref.frame.z - cp * sp.diff(theta,time) * ref.frame.z))] # stiffness/damping on pendulum


# --- Model and equations of motions
model = YAMSModel(ref=ref, bodies=bodies, body_loads=body_loads, 
                  coordinates=coordinates, speeds=speeds, kdeqsSubs=kdeqsSubs)
EOM=model.to_EOM()
print('Equations of motions')
EOM.EOM
EOM.mass_forcing_form()
print('Mass matrix:')
EOM.M
print('Forcing term:')
EOM.F

# --- KANE
print('Kanes full mass matrix')
MM = model.kane.mass_matrix_full
MM.simplify()
MM
print('Kanes full forcing')
FF = model.kane.forcing_full
FF.simplify()
FF

In [ ]:
# --- Simulation
vtime =  np.linspace(0.0, 15, 200)
u = {Fx:0}
p = {mc:20., mp:10., l:15, J_G:10, c:0, k:0, kp:0.0, cp:0.0, gravity:9.81}
q0 = {theta: 30*np.pi/180, x:0}
y = model.integrate(vtime, q0, p, u=u, M=MM, F=FF)
plt.plot(vtime, y);
plt.xlabel('Time [s]');
plt.legend(["${}$".format(sp.latex(c)) for c in model.q_full]);

In [ ]:
# --- Visualize simulation
cart.viz_opts={'type':'box','width':3., 'height':1., 'depth':0.3, 'color':'blue'}
pend.viz_opts={'type':'cylinder','radius':0.1, 'length':l,'normal':'y', 'color':'white'}
scene = model.getScene(rp=0.2, al=1)
scene.display()
#display(scene.display_jupyter()) # jupyter lab
#scene.display_ipython(); HTML(scene._html_widget.value)
